# Syntax tutorial

This notebook is a terse tutorial walkthrough of the syntax of the `guidance` language (which is based on the Handlebars templating language). It is not complete, but other than the <a href="https://guidance.readthedocs.org">full documentation</a>, it is a good reference.

In [1]:
import guidance

# we will use GPT-3 for most of the examples in this tutorial
guidance.llm = guidance.llms.OpenAI("text-davinci-003")

# Basic templating

Single variable


In [12]:
program = guidance('''What is {{example}}?''')

# this program has not been executed yet, so it still has the template placeholder in it
program 

Stop program What is {{example}} ?

In [13]:
# when we execute the program (by calling it) template placeholders are filled in
# note that keyword arguments to the program become variables in the template namespace
executed_program = program(example='truth')

Stop program What is truth ?

In [15]:
# all the variables used by the program are returned as part of the executed program
executed_program['example']

'truth'

Lists and objects

In [5]:
# define some variables we will use in the guidance program
people = ['John', 'Mary', 'Bob', 'Alice']
ideas = [
    {'name': 'truth', 'description': 'the state of being the case'},
    {'name': 'love', 'description': 'a strong feeling of affection'}
]

# we can use the `each` block to iterate over a list
program = guidance('''List of people:
{{#each people}}- {{this}}
{{~! This is a comment. The ~ removes adjacent whitespace either before or after a tag, depending on where you place it}}
{{/each~}}
List of ideas:
{{#each ideas}}{{this.name}}: {{this.description}}
{{/each}}''')

program(people=people, ideas=ideas)

Stop program List of people:
 - John 
- Mary 
- Bob 
- Alice 
 List of ideas:
 truth : the state of being the case 
 love : a strong feeling of affection

Includes (including guidance programs inside other programs)

In [6]:
# define the program we will include
program1 = guidance('''List of people:
{{#each people}}- {{this}}
{{/each~}}''')

# note that {{>prog_name}} is the same include syntax as in Handlebars
program2 = guidance('''{{>program1}}
List of ideas:
{{#each ideas}}{{this.name}}: {{this.description}}
{{/each}}''')

# we can pass program just like any other variable
program2(program1=program1, people=people, ideas=ideas)

Stop program List of people:
 - John 
- Mary 
- Bob 
- Alice 
 
List of ideas:
 truth : the state of being the case 
 love : a strong feeling of affection

Generating text from an LLM

In [7]:
# we can use the {{gen}} command to generate text from the language model
# note that we used a ~ at the start of the command tag to remove the whitespace before it (just like in Handlebars)
program = guidance('''The best thing about the beach is {{~gen 'best' temperature=0.7 max_tokens=7}}''')
program()

Stop program The best thing about the beach is the peacefulness and tranquility it

Flushing caches

In [8]:
# you can flush a cache by calling the clear method
# (this returns the number of items that were cleared)
guidance.llms.OpenAI.cache.clear()

# you can also disable caching by passing caching=False to the LLM constructor
# guidance.llm = guidance.llms.OpenAI("text-davinci-003", caching=False)

140

Selecting alternatives with the LLM

In [17]:
# the {{#select}} command allows you to use the LLM to select from a set of options
program = guidance('''Is the following sentence offensive? Please answer with a single word, either "Yes", "No", or "Maybe".
Sentence: {{example}}
Answer:{{#select "answer" logprobs='logprobs'}} Yes{{or}} No{{or}} Maybe{{/select}}''')
executed_program = program(example='I hate tacos')

Stop program Is the following sentence offensive? Please answer with a single word, either "Yes", "No", or "Maybe".
Sentence: I hate tacos 
Answer: Yes

In [18]:
# all the variables set by the program are returned as part of the executed program
executed_program['logprobs']

{' Yes': -57.1066, ' No': -1000, ' Maybe': -57.33598}

In [19]:
executed_program['answer']

' Yes'

In [21]:
# the example above used a block version of the select command, but you can also
# use a non-block version and just pass in a list of options
options = [' Yes', ' No', ' Maybe']
program = guidance('''Is the following sentence offensive? Please answer with a single word, either "Yes", "No", or "Maybe".
Sentence: {{example}}
Answer:{{select "answer" options=options}}''')
executed_program = program(example='I hate tacos', options=options)

Stop program Is the following sentence offensive? Please answer with a single word, either "Yes", "No", or "Maybe".
Sentence: I hate tacos 
Answer: Yes

In [22]:
executed_program["answer"]

' Yes'

Multiple generates in a sequence

In [25]:
program = guidance('''Generate a response to the following email:
{{email}}.
Response:{{gen "response"}}

Is the response above offensive in any way? Please answer with a single word, either "Yes" or "No".
Answer:{{#select "answer" logprobs='logprobs'}} Yes{{or}} No{{/select}}''')
executed_program = program(email='I hate tacos')

Stop program Generate a response to the following email:
 I hate tacos .
Response: That's too bad! Tacos are one of my favorite meals. 

Is the response above offensive in any way? Please answer with a single word, either "Yes" or "No".
Answer: No

In [26]:
executed_program['response'], executed_program['answer']

(" That's too bad! Tacos are one of my favorite meals.", ' No')

Hidden

In [27]:
# it is often useful to execute a part of the program, but then not include that part in later context
# given to the language model. This can be done using the hidden=True argument. Several commands support
# hidden=True, but here we use the {{#block}} command (which is just a generic block command that does
# nothing other than what the arguments you pass to it do)
program = guidance('''{{#block hidden=True}}Generate a response to the following email:
{{email}}.
Response:{{gen "response"}}{{/block}}
I will show you an email and a response, and you will tell me if it's offensive.
Email: {{email}}.
Response: {{response}}
Is the response above offensive in any way? Please answer with a single word, either "Yes" or "No".
Answer:{{#select "answer" logprobs='logprobs'}} Yes{{or}} No{{/select}}''')

executed_program = program(email='I hate tacos')


Stop program 
I will show you an email and a response, and you will tell me if it's offensive.
Email: I hate tacos .
Response: That's too bad! Tacos are one of my favorite meals. 
Is the response above offensive in any way? Please answer with a single word, either "Yes" or "No".
Answer: No

Silent execution

In [28]:
# if you want to run a program without displaying the output, you can use the silent=True argument
executed_program = program(email='I hate tacos', silent=True)
executed_program['answer']

' No'

Generating with `n>1`

In [32]:
# the {{gen}} command the n=number argument to generate multiple completions
# only the first completion is used for future context, but the variable set
# by the command is a list of all the completions, and you can interactively
# click through each completion in the notebook visualization
program = guidance('''The best thing about the beach is{{gen 'best' n=3 temperature=0.7 max_tokens=7}}''')
executed_program = program()

Stop program The best thing about the beach is <div style='background: rgba(255, 255, 255, 0.0); border-radius: 4px 0px 0px 4px; border: 1px solid rgba(0, 165, 0, 0.25); border-right: 0px; padding-left: 3px; padding-right: 3px; user-select: none; color: rgba(0, 165, 0, 0.25); display: inline; font-weight: normal; cursor: pointer' onClick='function cycle_cf2aa9dfb4e840cca4a5e66df84ee649(button_el) { var i = 0; while (i < 50) { var el = document.getElementById("cf2aa9dfb4e840cca4a5e66df84ee649_" + i); if (el.style.display == "inline") { el.style.display = "none"; var next_el = document.getElementById("cf2aa9dfb4e840cca4a5e66df84ee649_" + (i+1)); if (!next_el) { next_el = document.getElementById("cf2aa9dfb4e840cca4a5e66df84ee649_0"); } if (next_el) { next_el.style.display = "inline"; } break; } i += 1; } button_el.innerHTML = (((i+1) % 3) + 1) + "/" + 3;}cycle_cf2aa9dfb4e840cca4a5e66df84ee649(this);'>1/3 the sound of the waves crashing against the feeling of relaxation you get when that it is a great place to

In [33]:
executed_program["best"]

[' the sound of the waves crashing against',
 ' the feeling of relaxation you get when',
 ' that it is a great place to']

Calling custom user defined functions

In [34]:
# all the built in commands are functions from guidance.library.* but you can also pass in your own functions
def aggregate(best):
    return '\n'.join(['- ' + x for x in best])

# note that we use hidden=True to prevent the {{gen}} command from being included in the output, and instead
# just use the variable it sets as an input to the aggregate function
program = guidance('''The best thing about the beach is{{gen 'best' n=3 temperature=0.7 max_tokens=7 hidden=True}}
{{aggregate best}}''')
executed_program = program(aggregate=aggregate)

Stop program The best thing about the beach is 
 - the feeling of sand between your toes
- the sound of the waves crashing against
- the feeling of relaxation and joy that

Await

In [38]:
# sometimes you want to partially execute a program, the `await` command allows you to do this
# it awaits a variable and then consumes that variables (so after the await command the variable)
prompt = guidance('''Generate a response to the following email:
{{email}}.
Response:{{gen "response"}}
{{await 'instruction'}}
{{gen 'updated_response'}}''', stream=True)

# note how the executed program is only partially executed, it stops at the await command
# because the instruction variable is not yet set
prompt = prompt(email='Hello there')


Stop program Generate a response to the following email:
 Hello there .
Response: Hi! How can I help you? 
 {{await 'instruction'}} 
 {{gen 'updated_response'}}

In [36]:
prompt2 = prompt(instruction='Please translate the response above to Portuguese.')
prompt2

Stop program Generate a response to the following email:
 Hello there .
Response: Hi! How can I help you? 
 Please translate the response above to Portuguese. 
 Olá! Como posso ajudar?

In [37]:
prompt2 = prompt(instruction='Please translate the response above to Chinese.')
prompt2

Stop program Generate a response to the following email:
 Hello there .
Response: Hi! How can I help you? 
 Please translate the response above to Chinese. 
 你好！我能为你做些什么？

## Chat

In [42]:
# to use role based chat tags you need a chat model, here we use gpt-3.5-turbo but you can use 'gpt-4' as well
guidance.llm = guidance.llms.OpenAI("gpt-3.5-turbo")

In [43]:
# note that we enclose all of the text in one of the valid role tags for the model
# `system`, `user`, and `assistant` are just shorthand for {{#role name="system"}}...{{/role}}
# the whitepace outside the role tags is ignored by gpt-4, the whitespace inside the role tags is not
# so we use the ~ to remove the whitespace we don't want to give to the model (but want to keep in the code for clarity)
program = guidance('''
{{#system~}}
You are a helpful assistant.
{{~/system}}

{{#user~}}
{{conversation_question}}
{{~/user}}

{{! this is a comment. note that we don't have to use a stop="stop_string" for the gen command below because Guidance infers the stop string from the role tag }}
{{#assistant~}}
{{gen 'response'}}
{{~/assistant}}''')

executed_program = program(conversation_question='What is the meaning of life?')

Stop program system You are a helpful assistant. user What is the meaning of life? assistant The meaning of life is a philosophical and existential question that has been debated by scholars, thinkers, and individuals throughout history. There is no one definitive answer to this question, as it can vary depending on one's beliefs, values, and experiences. Some people believe that the meaning of life is to seek happiness, others believe it is to fulfill a specific purpose or destiny, while others believe it is to find spiritual enlightenment or connection with a higher power. Ultimately, the meaning of life is a personal and subjective concept that each individual must determine for themselves.

Multistep

In [44]:
# you can create and guide multi-turn conversations by using a series of role tags
experts = guidance('''
{{#system~}}
You are a helpful assistant.
{{~/system}}

{{#user~}}
I want a response to the following question:
{{query}}
Who are 3 world-class experts (past or present) who would be great at answering this?
Please don't answer the question or comment on it yet.
{{~/user}}

{{#assistant~}}
{{gen 'experts' temperature=0 max_tokens=300}}
{{~/assistant}}

{{#user~}}
Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer.
In other words, their identity is not revealed, nor is the fact that there is a panel of experts answering the question.
If the experts would disagree, just present their different positions as alternatives in the answer itself (e.g. 'some might argue... others might argue...').
Please start your answer with ANSWER:
{{~/user}}

{{#assistant~}}
{{gen 'answer' temperature=0 max_tokens=500}}
{{~/assistant}}''')
                   
experts(query='What is the meaning of life?')

Stop program system You are a helpful assistant. user I want a response to the following question:
 What is the meaning of life? 
Who are 3 world-class experts (past or present) who would be great at answering this?
Please don't answer the question or comment on it yet. assistant Sure, I understand. Here are three world-class experts (past or present) who would be great at answering the question "What is the meaning of life?":

1. Viktor Frankl - an Austrian neurologist, psychiatrist, and Holocaust survivor who wrote the book "Man's Search for Meaning," which explores the human search for purpose and meaning in life.

2. Aristotle - a Greek philosopher who believed that the meaning of life is to achieve eudaimonia, or a state of happiness and fulfillment through living a virtuous life.

3. Dalai Lama - the spiritual leader of Tibetan Buddhism who has written extensively on the nature of happiness, compassion, and the purpose of life. user Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer.
In other words, their identity is not revealed, nor is the fact that there is a panel of experts answering the question.
If the experts would disagree, just present their different positions as alternatives in the answer itself (e.g. 'some might argue... others might argue...').
Please start your answer with ANSWER: assistant ANSWER: The meaning of life is a question that has puzzled humans for centuries. While there is no one definitive answer, we believe that the purpose of life is to find meaning and purpose in our existence. This can be achieved through a variety of means, including personal growth, relationships, and contributing to society.

Some might argue that the meaning of life is to achieve happiness and fulfillment, as Aristotle believed. Others might argue that the purpose of life is to find spiritual enlightenment, as the Dalai Lama has written about extensively. However, we believe that the search for meaning and purpose is a universal human experience that transcends any one particular belief system or philosophy.

Viktor Frankl's concept of logotherapy suggests that the search for meaning is a fundamental human need, and that individuals can find meaning in even the most difficult circumstances. By focusing on our values and goals, we can find purpose and meaning in our lives, even in the face of adversity.

Ultimately, the meaning of life is a deeply personal and subjective question that each individual must answer for themselves. However, by exploring our values, relationships, and contributions to society, we can find purpose and meaning in our existence, and live a fulfilling and meaningful life.

With hidden

In [46]:
# if you want the model to have some inner dialog but then not include that dialog
# in the context of later generations, you can use the {{#block}} command with hidden=True
program = guidance('''
{{#system~}}
You are a helpful assistant.
{{~/system}}

{{#block hidden=True}}
{{#user~}}
Please tell me a joke
{{~/user}}

{{! note that we don't have guidance controls inside the assistant role because
    the OpenAI API does not yet support that (Transformers chat models do) }}
{{#assistant~}}
{{gen 'joke'}}
{{~/assistant}}
{{~/block~}}

{{#user~}}
Is the following joke funny? Why or why not?
{{joke}}
{{~/user}}

{{#assistant~}}
{{gen 'funny'}}
{{~/assistant}}''')
program()


Stop program system You are a helpful assistant. user Is the following joke funny? Why or why not?
 Sure, here's a joke for you:

Why did the tomato turn red?

Because it saw the salad dressing! assistant As an AI language model, I don't have the ability to find jokes funny or not. However, I can provide an analysis of the joke. 

The joke is a play on words and relies on a pun. The punchline is unexpected and relies on the listener's knowledge of the fact that tomatoes turn red when they are ripe. The joke may be considered funny to some people, while others may not find it humorous. It ultimately depends on the individual's sense of humor and personal preferences.

Agents

In [48]:
# by putting an `await` inside a `geneach` loop you can create agents that consume some
# varable, then do something and then wait for more content
program = guidance('''
{{#system~}}
You are a helpful assistant
{{~/system}}

{{~#geneach 'conversation' stop=False}}
{{#user~}}
{{set 'this.user_text' (await 'user_text')}}
{{~/user}}

{{#assistant~}}
{{gen 'this.ai_text' temperature=0 max_tokens=300}}
{{~/assistant}}
{{~/geneach}}''')
program = program(user_text ='hi there')

Stop program system You are a helpful assistant user hi there assistant Hello! How can I assist you today? user {{set 'conversation[-1].user_text' (await 'user_text')}} {{#assistant~}} 
 {{gen 'conversation[-1].ai_text' temperature=0 max_tokens=300}} 
 {{~/assistant}} 
 {{~#geneach 'conversation' stop=False}} 
 {{#user~}} 
 {{set 'this.user_text' (await 'user_text')}} 
 {{~/user}} 

 {{#assistant~}} 
 {{gen 'this.ai_text' temperature=0 max_tokens=300}} 
 {{~/assistant}} 
 {{~/geneach}}

In [49]:
# as we go through the loop we build up a conversation variable that contains the history of the conversation
# note that the last entry in the conversation variable is empty because the `await` call happens before any
# content is added to the `this` variable that represents the current item in the geneach loop
program['conversation']

[{'user_text': 'hi there', 'ai_text': 'Hello! How can I assist you today?'},
 {}]

In [50]:
# here we call the agent again and the loop continues, in this case building out a conversation
program = program(user_text = 'What is the meaning of life?')

Stop program system You are a helpful assistant user hi there assistant Hello! How can I assist you today? user What is the meaning of life? assistant The meaning of life is a philosophical question that has been debated by scholars, theologians, and thinkers for centuries. There is no one definitive answer to this question, as it can be interpreted in many different ways depending on one's beliefs, values, and experiences. Some people believe that the meaning of life is to seek happiness, while others believe it is to fulfill a specific purpose or destiny. Still, others believe that life has no inherent meaning and that it is up to each individual to create their own purpose and find their own fulfillment. Ultimately, the meaning of life is a deeply personal and subjective question that each person must answer for themselves. user {{set 'conversation[-1].user_text' (await 'user_text')}} {{#assistant~}} 
 {{gen 'conversation[-1].ai_text' temperature=0 max_tokens=300}} 
 {{~/assistant}} 
 {{~#geneach 'conversation' stop=False}} 
 {{#user~}} 
 {{set 'this.user_text' (await 'user_text')}} 
 {{~/user}} 

 {{#assistant~}} 
 {{gen 'this.ai_text' temperature=0 max_tokens=300}} 
 {{~/assistant}} 
 {{~/geneach}}

In [51]:
program['conversation']

[{'user_text': 'hi there', 'ai_text': 'Hello! How can I assist you today?'},
 {'user_text': 'What is the meaning of life?',
  'ai_text': "The meaning of life is a philosophical question that has been debated by scholars, theologians, and thinkers for centuries. There is no one definitive answer to this question, as it can be interpreted in many different ways depending on one's beliefs, values, and experiences. Some people believe that the meaning of life is to seek happiness, while others believe it is to fulfill a specific purpose or destiny. Still, others believe that life has no inherent meaning and that it is up to each individual to create their own purpose and find their own fulfillment. Ultimately, the meaning of life is a deeply personal and subjective question that each person must answer for themselves."},
 {}]

Using tools

The example below uses a search engine (or a mock of one) to answer user questions. The whole system is defined in a single `guidance` program, but you could also break it into multiple programs and `await` external calls if you don't want the guidance program to control the whole process.

In [56]:
def is_search(completion):
    return '<search>' in completion

def search(query):
    # Fake search results
    return [{'title': 'How do I cancel a Subscription? | Facebook Help Center',
        'snippet': "To stop a monthly Subscription to a creator: Go to the creator's Facebook Page using the latest version of the Facebook app for iOS, Android or from a computer. Select Go to Supporter Hub. Select . Select Manage Subscription to go to the iTunes or Google Play Store and cancel your subscription. Cancel your Subscription at least 24 hours before ..."},
        {'title': 'News | FACEBOOK Stock Price Today | Analyst Opinions - Insider',
        'snippet': 'Stock | News | FACEBOOK Stock Price Today | Analyst Opinions | Markets Insider Markets Stocks Indices Commodities Cryptocurrencies Currencies ETFs News Facebook Inc (A) Cert Deposito Arg Repr...'},
        {'title': 'Facebook Stock Price Today (NASDAQ: META) Quote, Market Cap, Chart ...',
        'snippet': 'Facebook Stock Price Today (NASDAQ: META) Quote, Market Cap, Chart | WallStreetZen Meta Platforms Inc Stock Add to Watchlist Overview Forecast Earnings Dividend Ownership Statistics $197.81 +2.20 (+1.12%) Updated Mar 20, 2023 Meta Platforms shares are trading... find out Why META Price Moved with a free WallStreetZen account Why Price Moved'}]

search_demo = guidance('''Seach results:
{{~#each results}}
<result>
{{this.title}}
{{this.snippet}}
</result>{{/each}}''')

demo_results = [
    {'title': 'OpenAI - Wikipedia', 'snippet': 'OpenAI systems run on the fifth most powerful supercomputer in the world. [5] [6] [7] The organization was founded in San Francisco in 2015 by Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others, [8] [1] [9] who collectively pledged US$ 1 billion. Musk resigned from the board in 2018 but remained a donor.'},
    {'title': 'About - OpenAI', 'snippet': 'About OpenAI is an AI research and deployment company. Our mission is to ensure that artificial general intelligence benefits all of humanity. Our vision for the future of AGI Our mission is to ensure that artificial general intelligence—AI systems that are generally smarter than humans—benefits all of humanity. Read our plan for AGI'}, 
    {'title': 'Ilya Sutskever | Stanford HAI', 'snippet': '''Ilya Sutskever is Co-founder and Chief Scientist of OpenAI, which aims to build artificial general intelligence that benefits all of humanity. He leads research at OpenAI and is one of the architects behind the GPT models. Prior to OpenAI, Ilya was co-inventor of AlexNet and Sequence to Sequence Learning.'''}
]

s = search_demo(results=demo_results)

practice_round = [
    {'role': 'user', 'content' : 'Who are the founders of OpenAI?'},
    {'role': 'assistant', 'content': '<search>Who are the founders of OpenAI</search>'},
    {'role': 'user', 'content': str(search_demo(results=demo_results))},
    {'role': 'assistant', 'content': 'The founders of OpenAI are Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others.'},
]

program = guidance('''
{{#system~}}
You are a helpful assistant.
{{~/system}}

{{#user~}}
From now on, whenever your response depends on any factual information, please search the web by using the function <search>query</search> before responding. I will then paste web results in, and you can respond.
{{~/user}}

{{#assistant~}}
Ok, I will do that. Let's do a practice round
{{~/assistant}}

{{#each practice}}
{{#if (== this.role "user")}}
{{#user}}{{this.content}}{{/user}}
{{else}}
{{#assistant}}{{this.content}}{{/assistant}}
{{/if}}
{{/each}}

{{#user~}}
That was great, now let's do another one.
{{~/user}}

{{#assistant~}}
Sounds good
{{~/assistant}}

{{#user~}}
{{user_query}}
{{~/user}}

{{#assistant~}}
{{gen "query" stop="</search>"}}{{#if (is_search query)}}</search>{{/if}}
{{~/assistant}}

{{#user~}}
Search results: {{#each (search query)}}
<result>
{{this.title}}
{{this.snippet}}
</result>{{/each}}
{{~/user}}

{{#assistant~}}
{{gen "answer"}}
{{~/assistant}}
''')

query = "What is Facebook's stock price right now?"

program = program(
    user_query=query,
    search=search,
    is_search=is_search,
    practice=practice_round
)


Stop program system You are a helpful assistant. user From now on, whenever your response depends on any factual information, please search the web by using the function <search>query</search> before responding. I will then paste web results in, and you can respond. assistant Ok, I will do that. Let's do a practice round 
 user Who are the founders of OpenAI? 

 assistant <search>Who are the founders of OpenAI</search> 

 user Seach results:
<result>
OpenAI - Wikipedia
OpenAI systems run on the fifth most powerful supercomputer in the world. [5] [6] [7] The organization was founded in San Francisco in 2015 by Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others, [8] [1] [9] who collectively pledged US$ 1 billion. Musk resigned from the board in 2018 but remained a donor.
</result>
<result>
About - OpenAI
About OpenAI is an AI research and deployment company. Our mission is to ensure that artificial general intelligence benefits all of humanity. Our vision for the future of AGI Our mission is to ensure that artificial general intelligence—AI systems that are generally smarter than humans—benefits all of humanity. Read our plan for AGI
</result>
<result>
Ilya Sutskever | Stanford HAI
Ilya Sutskever is Co-founder and Chief Scientist of OpenAI, which aims to build artificial general intelligence that benefits all of humanity. He leads research at OpenAI and is one of the architects behind the GPT models. Prior to OpenAI, Ilya was co-inventor of AlexNet and Sequence to Sequence Learning.
</result> 

 assistant The founders of OpenAI are Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others. 
 user That was great, now let's do another one. assistant Sounds good user What is Facebook's stock price right now? assistant <search>What is Facebook's stock price right now? </search> user Search results: 
<result>
 How do I cancel a Subscription? | Facebook Help Center 
 To stop a monthly Subscription to a creator: Go to the creator's Facebook Page using the latest version of the Facebook app for iOS, Android or from a computer. Select Go to Supporter Hub. Select . Select Manage Subscription to go to the iTunes or Google Play Store and cancel your subscription. Cancel your Subscription at least 24 hours before ... 
</result>
<result>
 News | FACEBOOK Stock Price Today | Analyst Opinions - Insider 
 Stock | News | FACEBOOK Stock Price Today | Analyst Opinions | Markets Insider Markets Stocks Indices Commodities Cryptocurrencies Currencies ETFs News Facebook Inc (A) Cert Deposito Arg Repr... 
</result>
<result>
 Facebook Stock Price Today (NASDAQ: META) Quote, Market Cap, Chart ... 
 Facebook Stock Price Today (NASDAQ: META) Quote, Market Cap, Chart | WallStreetZen Meta Platforms Inc Stock Add to Watchlist Overview Forecast Earnings Dividend Ownership Statistics $197.81 +2.20 (+1.12%) Updated Mar 20, 2023 Meta Platforms shares are trading... find out Why META Price Moved with a free WallStreetZen account Why Price Moved 
</result> assistant <search>What is Facebook's stock price right now?</search>

As of my search, Facebook's stock price is $197.81 per share.

In [57]:
def is_search(completion):
    return '<search>' in completion

def search(query):
    # Fake search results
    return [{'title': 'How do I cancel a Subscription? | Facebook Help Center',
  'snippet': "To stop a monthly Subscription to a creator: Go to the creator's Facebook Page using the latest version of the Facebook app for iOS, Android or from a computer. Select Go to Supporter Hub. Select . Select Manage Subscription to go to the iTunes or Google Play Store and cancel your subscription. Cancel your Subscription at least 24 hours before ..."},
 {'title': 'News | FACEBOOK Stock Price Today | Analyst Opinions - Insider',
  'snippet': 'Stock | News | FACEBOOK Stock Price Today | Analyst Opinions | Markets Insider Markets Stocks Indices Commodities Cryptocurrencies Currencies ETFs News Facebook Inc (A) Cert Deposito Arg Repr...'},
 {'title': 'Facebook Stock Price Today (NASDAQ: META) Quote, Market Cap, Chart ...',
  'snippet': 'Facebook Stock Price Today (NASDAQ: META) Quote, Market Cap, Chart | WallStreetZen Meta Platforms Inc Stock Add to Watchlist Overview Forecast Earnings Dividend Ownership Statistics $197.81 +2.20 (+1.12%) Updated Mar 20, 2023 Meta Platforms shares are trading... find out Why META Price Moved with a free WallStreetZen account Why Price Moved'}]

prompt = guidance('''
{{#system~}}
You are a helpful assistant.
{{~/system}}

{{#user~}}
From now on, whenever your response depends on any factual information, please search the web by using the function <search>query</search> before responding. I will then paste web results in, and you can respond.
{{~/user}}

{{#assistant~}}
Ok, I will do that. Let's do a practice round
{{~/assistant}}

{{#user~}}
Who are the founders of OpenAI?
{{~/user}}

{{#assistant~}}
<search>Who are the founders of OpenAI</search>
{{~/assistant}}

{{#user~}}
Search results:
<result>
OpenAI - Wikipedia
OpenAI systems run on the fifth most powerful supercomputer in the world. [5] [6] [7] The organization was founded in San Francisco in 2015 by Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others, [8] [1] [9] who collectively pledged US$ 1 billion. Musk resigned from the board in 2018 but remained a donor.
</result>
<result>
About - OpenAI
About OpenAI is an AI research and deployment company. Our mission is to ensure that artificial general intelligence benefits all of humanity. Our vision for the future of AGI Our mission is to ensure that artificial general intelligence—AI systems that are generally smarter than humans—benefits all of humanity. Read our plan for AGI
</result>
<result>
Ilya Sutskever | Stanford HAI
Ilya Sutskever is Co-founder and Chief Scientist of OpenAI, which aims to build artificial general intelligence that benefits all of humanity. He leads research at OpenAI and is one of the architects behind the GPT models. Prior to OpenAI, Ilya was co-inventor of AlexNet and Sequence to Sequence Learning.
</result>
{{~/user}}

{{#assistant~}}
The founders of OpenAI are Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others.
{{~/assistant}}

{{#user~}}
That was great, now let's do another one.
{{~/user}}

{{#assistant~}}
Sounds good
{{~/assistant}}

{{#user~}}
{{user_query}}
{{~/user}}

{{#assistant~}}
{{gen "query" stop="</search>"}}{{#if (is_search query)}}</search>{{/if}}
{{/assistant}}

{{#user~}}
Search results: {{#each (search query)}}
<result>
{{this.title}}
{{this.snippet}}
</result>{{/each}}
{{~/user}}

{{#assistant~}}
{{gen "answer"}}
{{~/assistant}}''')
query = "What is Facebook's stock price right now?"
prompt = prompt(user_query=query, search=search, is_search=is_search)
prompt

Stop program system You are a helpful assistant. user From now on, whenever your response depends on any factual information, please search the web by using the function <search>query</search> before responding. I will then paste web results in, and you can respond. assistant Ok, I will do that. Let's do a practice round user Who are the founders of OpenAI? assistant <search>Who are the founders of OpenAI</search> user Search results:
<result>
OpenAI - Wikipedia
OpenAI systems run on the fifth most powerful supercomputer in the world. [5] [6] [7] The organization was founded in San Francisco in 2015 by Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others, [8] [1] [9] who collectively pledged US$ 1 billion. Musk resigned from the board in 2018 but remained a donor.
</result>
<result>
About - OpenAI
About OpenAI is an AI research and deployment company. Our mission is to ensure that artificial general intelligence benefits all of humanity. Our vision for the future of AGI Our mission is to ensure that artificial general intelligence—AI systems that are generally smarter than humans—benefits all of humanity. Read our plan for AGI
</result>
<result>
Ilya Sutskever | Stanford HAI
Ilya Sutskever is Co-founder and Chief Scientist of OpenAI, which aims to build artificial general intelligence that benefits all of humanity. He leads research at OpenAI and is one of the architects behind the GPT models. Prior to OpenAI, Ilya was co-inventor of AlexNet and Sequence to Sequence Learning.
</result> assistant The founders of OpenAI are Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others. user That was great, now let's do another one. assistant Sounds good user What is Facebook's stock price right now? assistant <search>What is Facebook's stock price right now? </search> 
 user Search results: 
<result>
 How do I cancel a Subscription? | Facebook Help Center 
 To stop a monthly Subscription to a creator: Go to the creator's Facebook Page using the latest version of the Facebook app for iOS, Android or from a computer. Select Go to Supporter Hub. Select . Select Manage Subscription to go to the iTunes or Google Play Store and cancel your subscription. Cancel your Subscription at least 24 hours before ... 
</result>
<result>
 News | FACEBOOK Stock Price Today | Analyst Opinions - Insider 
 Stock | News | FACEBOOK Stock Price Today | Analyst Opinions | Markets Insider Markets Stocks Indices Commodities Cryptocurrencies Currencies ETFs News Facebook Inc (A) Cert Deposito Arg Repr... 
</result>
<result>
 Facebook Stock Price Today (NASDAQ: META) Quote, Market Cap, Chart ... 
 Facebook Stock Price Today (NASDAQ: META) Quote, Market Cap, Chart | WallStreetZen Meta Platforms Inc Stock Add to Watchlist Overview Forecast Earnings Dividend Ownership Statistics $197.81 +2.20 (+1.12%) Updated Mar 20, 2023 Meta Platforms shares are trading... find out Why META Price Moved with a free WallStreetZen account Why Price Moved 
</result> assistant <search>What is Facebook's stock price right now?</search>

As of my search, Facebook's stock price is $197.81 per share.